In [1]:
import re

import jieba.analyse
import os

from tortoise import Tortoise
from config import TORTOISE_ORM
from entity.db_entity import *

await Tortoise.init(TORTOISE_ORM)

In [2]:
from typing import Dict, List

In [3]:
all_course_id = [i[:-4] for i in os.listdir("./texts")]
course_data = [[a,b,re.findall(r"daxuexi/(.+?)/", d)[0]] for a,b,d in await Course.filter(id__in=all_course_id).values_list("id", "title", "uri")]

In [4]:
xxx = ['请', '不吝', '点赞', ' ', '订阅', '转发', '打赏', '支持', '明镜', '与', '点点', '栏目']

In [5]:
for i in range(len(course_data)):
    with open("./texts/"+course_data[i][0]+".txt",'r',encoding='utf-8') as f:
        text = f.read()
    result:list = jieba.analyse.textrank(text, topK=20, withWeight=True)
    for k, v in result:
        if k in xxx:
            result.remove((k, v))
    result = [[k, int(v*10)] for k, v in result[:10]]
    course_data[i].append(result)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lid\AppData\Local\Temp\jieba.cache
Loading model cost 0.580 seconds.
Prefix dict has been built successfully.


In [6]:
course_data[0]

['C0001',
 '第八季第一期',
 'daxuexi8q1',
 [['疫情', 10],
  ['防控', 5],
  ['力量', 5],
  ['工作', 4],
  ['綏靖', 3],
  ['打好', 3],
  ['发挥', 3],
  ['行动', 3],
  ['人民', 3],
  ['医院', 3]]]

In [7]:
cn2code = lambda string: "".join(re.findall(r"\\\\u(.{4})", str(string.encode('unicode-escape'))))

只在某一期出现过的词同一个颜色

多期出现的词按出现次数分颜色

课程单独一个颜色

In [8]:
nodes = []
links = []
node_group = {}
for course in course_data:
    nodes.append({"id":course[2], "name":course[1], "group":course[2]})

    for i in course[-1]:
        temp = {"id":cn2code(i[0]), "name":i[0], "group":0}

        if i[0] in node_group.keys():
            if type(node_group[i[0]]) is int:
                node_group[i[0]] += 1
            else:
                node_group[i[0]] = 2
        else:
            node_group[i[0]] = course[2]

        if temp not in nodes:
            nodes.append(temp)
        links.append({"source":course[2], "target":temp["id"], "value":i[1]})

for i in range(len(nodes)):
    if nodes[i]["name"] in node_group.keys():
        nodes[i]["group"] = node_group[nodes[i]["name"]]

In [11]:
# {"nodes":[{"id":"", "name":"", "feature":""}],
#  "links":[{"source":"", "target":"", "value":""}]}
final = {"nodes":nodes, "links":links}
final

{'nodes': [{'id': 'daxuexi8q1', 'name': '第八季第一期', 'group': 'daxuexi8q1'},
  {'id': '75ab60c5', 'name': '疫情', 'group': 16},
  {'id': '963263a7', 'name': '防控', 'group': 9},
  {'id': '529b91cf', 'name': '力量', 'group': 3},
  {'id': '5de54f5c', 'name': '工作', 'group': 5},
  {'id': '7d8f9756', 'name': '綏靖', 'group': 'daxuexi8q1'},
  {'id': '6253597d', 'name': '打好', 'group': 'daxuexi8q1'},
  {'id': '53d16325', 'name': '发挥', 'group': 'daxuexi8q1'},
  {'id': '884c52a8', 'name': '行动', 'group': 'daxuexi8q1'},
  {'id': '4eba6c11', 'name': '人民', 'group': 25},
  {'id': '533b9662', 'name': '医院', 'group': 2},
  {'id': 'daxuexi8w2', 'name': '第八季第二期', 'group': 'daxuexi8w2'},
  {'id': '5b664e60', 'name': '学习', 'group': 18},
  {'id': '7fa44f17', 'name': '群众', 'group': 6},
  {'id': '963b51fb6218', 'name': '阻击战', 'group': 'daxuexi8w2'},
  {'id': '603b4e668bb0', 'name': '总书记', 'group': 16},
  {'id': '51714ea7515a5458', 'name': '共产党员', 'group': 2},
  {'id': '5e948be5', 'name': '应该', 'group': 'daxuexi8w2'},
  {

In [ ]:
import json
with open("../3d-force-graph/example/datasets/test.json", "w", encoding="utf-8") as f:
    json.dump(final, f)